In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import cifar100
from keras.models import Sequential ,Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import SGD, Adam
from keras.applications import VGG16

Using TensorFlow backend.


### Liberar la memoria de la GPU

In [2]:
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

###  Mirar las GPU's disponibles

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14485351556778813950
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6671596257
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8956029428906323406
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


In [4]:
batch_size = 100
num_classes = 100
epochs = 15

In [5]:
(xt,yt),(xtest,ytest) = cifar100.load_data()

_,filas, columnas, canales = xt.shape

xt = xt.astype('float32')
xtest = xtest.astype('float32')

xt = xt / 255
xtest = xtest / 255

yt = keras.utils.to_categorical(yt, num_classes)
ytest = keras.utils.to_categorical(ytest, num_classes)

In [6]:
Basica = 1

In [7]:
# Topología más sencilla
# No es capaz de procesar adecudamente los datos, no puede aprender
if Basica == 1:
    Entradas = Input(shape=(filas, columnas, canales))

    # Capa convolucional a las entradas, 64 filtros con tamaño de 3x3
    x = Conv2D(64, kernel_size=(3,3), activation='relu')(Entradas)  

    # Capa convolucional a la salida de la primera convolución, 128 filtros con tamaño de 3x3
    x = Conv2D(128, kernel_size=(3,3), activation='relu')(x) 
    x = MaxPooling2D(pool_size=(2,2))(x) # Reducir el tamaño de la salida y obtener las características más importantes
    x = Flatten()(x)  # Reducir a una dimensión
    x = Dense(100, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    
# Bancos de Convoluciones    
if Basica == 0:
    Entradas = Input(shape=(filas, columnas, canales))
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(Entradas)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
modelo = Model(inputs=Entradas, outputs=x)
modelo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2508900   
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
Total para

In [9]:
descenso_gradiente_estocastico = Adam(lr=0.001, beta_1=0.9, beta_2=0.9) #SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
modelo.compile(loss=keras.losses.categorical_crossentropy, optimizer=descenso_gradiente_estocastico, metrics=['categorical_accuracy'])

In [10]:
modelo.fit(xt, yt, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(xtest, ytest))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 19s 373us/step - loss: 3.8678 - categorical_accuracy: 0.1193 - val_loss: 3.4550 - val_categorical_accuracy: 0.1893
Epoch 2/15
38800/50000 [======================>.......] - ETA: 3s - loss: 3.1835 - categorical_accuracy: 0.2336

KeyboardInterrupt: 

In [11]:
puntuacion = modelo.evaluate(xtest, ytest, verbose=1)
print(puntuacion)

10000/10000 [==============================] - 1s 89us/step
[1.4627258655548097, 0.5409]
